In [30]:
# For modules loading
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
from tqdm import tqdm
import regex as re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
#from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, T5ForConditionalGeneration
import torch
from itertools import cycle
import os 
from transformers import pipeline

from rouge_score import rouge_scorer

# For BM25
import src.bm25_IR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generate retrieval corpus

In [31]:
path_adam = '/Users/adamwagnerhoegh/Documents/Legal data/domsdatabasen.retsinformation_newer.json'
path_asger = "/Users/asgerkromand/Library/CloudStorage/OneDrive-UniversityofCopenhagen/0. SDS/1 deep learning and nlp/ANLPDP_exam/data/domsdatabasen.retsinformation_newer.json"
path_andreas = '' #missing

# Define a function that can cycle through paths the above paths try them out, and yield the path
def path():
    paths = cycle([path_adam, path_asger, path_andreas])
    for path in paths:
        if path != '':
            try:
                with open(path, 'r') as f:
                    data = json.load(f)
                return data
            except:
                pass
        else:
            raise FileNotFoundError('No path to data found')

retsinfo = path()
    
rag_list = []
idx = 0
for lov in tqdm(retsinfo):
    for kapitel in lov['kapitler']:
        lov_navn = lov['shortName']
        for paragraffer in kapitel['paragraffer']:
            temp_paragraf_dict = {}
            temp_paragraf_dict['paragraf_nr'] = paragraffer['nummer']
            temp_paragraf_dict['lovnavn'] = lov_navn
            temp_paragraf_list = []
            for styk in paragraffer['stk']:
                temp_paragraf_list.append(styk['tekst'])
            temp_paragraf_dict['text'] = ' '.join(temp_paragraf_list)
            rag_list.append(temp_paragraf_dict)

with open("rag_list.txt", "w") as file:
    for item in rag_list:
        file.write(f"{item}\n")

100%|██████████| 1637/1637 [00:00<00:00, 27802.83it/s]


## Generate dev set

In [32]:
# load excel files in dev set folder
import os

dev_set_folder = "devset"

dfs = []
for file in os.listdir(dev_set_folder):
    if file.endswith(".xlsx"):
        df = pd.read_excel(os.path.join(dev_set_folder, file))
        dfs.append(df)

# merge all excel
dev_set = pd.concat(dfs, ignore_index=True)

# add csv
rag_batch_1_with_qa = pd.read_csv("devset/rag_batch_1_with_qa.csv", sep=",").iloc[:, 1:].dropna()
rag_batch_1_with_qa.columns = dev_set.columns
dev_set = pd.concat([dev_set, rag_batch_1_with_qa], ignore_index=True)

# output dev set
dev_set.to_csv("devset/dev_set.csv", index=False)

In [33]:
dev_set[0:33]

,"question, str","answer, str","text, str","pnumber, str","law number, str"
0,"Hvad har ejeren af en ejerlejlighed, sammen me...","Grunden, fælles bestanddele og tilbehør.",Ejeren af en ejerlejlighed har sammen med andr...,3,LOV nr 908 af 18/06/2020
1,Hvem fastsætter eller aftaler bestemmelser om ...,Finansministeren fastsætter eller aftaler best...,Højskolen skal følge de af finansministeren fa...,30,LBK nr 780 af 08/08/2019
2,Hvad skal Beskæftigelsesministeriet og Finanst...,Den indsendte årsrapport skal i det mindste in...,Uden ugrundet ophold efter repræsentantskabets...,25 l,LBK nr 1110 af 10/10/2014
3,Hvor mange procent må kapitalandele i og lån y...,Kapitalandele i og lån ydet til en virksomhed ...,Følgende grænser for Arbejdsmarkedets Tillægsp...,26 e,LBK nr 1110 af 10/10/2014
4,Hvad er en betingelse for retten til jobpræmie?,Det er en betingelse for retten til jobpræmie ...,"Det er en betingelse for retten til jobpræmie,...",9,LOV nr 287 af 29/03/2017
5,Hvem kan tillade de ifølge deklaration eller v...,"For ejendomme, der tilhører byggeforeninger (b...","For ejendomme, der tilhører private kan ejeren...",2,LBK nr 978 af 19/10/2009'
6,Hvem kan på baggrund af en evaluering fastsætt...,Undervisningsministeren kan på baggrund af en...,Undervisningsministeren kan fastsætte regler o...,10,LBK nr 600 af 21/05/2019
7,Hvornår kan Beskæftigelsesministeren efter for...,Hvis selskabet groft eller gentagne gange tils...,"Tilsidesætter selskabet nævnt i § 10, stk. 1, ...",25,LOV nr 994 af 30/08/2015
8,Hvornår kan brevåbning og brevstandsning foret...,Hvis der foreligger en særligt bestyrket mista...,Indgreb i meddelelseshemmeligheden må kun fore...,813,LOV nr 964 af 26/06/2020
9,Hvem afgør om en bolig skal brandsikres?,Boligtilsynet.,"Ejeren af en beboelsesbygning, som efter medde...",NaN,LBK nr 1710 af 16/12/2010


## Vectorize retrieval corpus

### Sparse retrieval TF-IDF

In [14]:
rag_list2 = rag_list

def preprocess(rag_list):
    # extract and preprocess text
    corpus = [item['text'] for item in rag_list]
    corpus = [re.sub('\\s{2,}', ' ', 
                     re.sub('\\W|[0-9]|§', ' ',
                           item.lower())) for item in corpus]

    # remove stopwords
    #nltk.download('punkt')
    stop_words = set(stopwords.words('danish'))
    corpus = [' '.join(word for word in text.split() 
                      if word not in stop_words) for text in tqdm(corpus)]
    
    return corpus

corpus = preprocess(rag_list2)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

100%|██████████| 42593/42593 [00:00<00:00, 64028.83it/s]


### Dense retrieval

In [5]:
## WRITE LATER

## RAG retriever

### Sparse retrieval pipeline for TF-IDF


In [15]:
def sparse_retrieval(question, corpus_embeddings=X, corpus=rag_list, vectorizer=vectorizer, k=1, max_tokens=800):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    """

    # preprocess and vectorize question
    question_processed = [re.sub('\\s{2,}', ' ', 
                               re.sub('\\W|[0-9]|§', ' ',
                                     question.lower()))]
    
    # remove stopwords
    stop_words = set(stopwords.words('danish'))
    question_processed = [' '.join(word for word in text.split() 
                                 if word not in stop_words) for text in question_processed]
    
    # embed question
    question_vector = vectorizer.transform(question_processed)

    # calculate cosine similarity
    sparse_retrieval = corpus_embeddings.dot(question_vector.T).toarray()

    # get top k paragraphs
    top_k = np.argsort(sparse_retrieval.flatten())[-k:]

    # truncate context to approximate token limit
    context = '\n'.join([corpus[i]['text'] for i in top_k])
    context_words = context.split()[:max_tokens]
    return ' '.join(context_words)

    return context

# check if it works using a random question from the dev set
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(random_question, '\n')
sparse_retrieval(random_question, X)

Hvem fastsættes bestemmelser om indretning og vedligeholdelse af tjenesteboliger? 



'Såfremt det af hensyn til udførelsen af tjenestepligterne skønnes nødvendigt at knytte tjenestebolig til stillingen, har tjenestemanden pligt til at bebo denne, så længe han beklæder stillingen, og til derefter at fraflytte den. Bestemmelser om indretning, vedligeholdelse og benyttelse af tjenesteboliger og om ind- og udflytning m.v. af sådanne boliger fastsættes af finansministeren efter indhentet udtalelse fra Lønningsrådet.'

### Sparse retrieval pipeline for BM25

In [16]:
# Import bm25
import src.bm25_IR as bm25

**Load in dev data and tokenize**

In [ ]:
# Path to the dev data
dev_data_path = "./devset/dev_set.csv"

# Import the data
bm25_data = bm25.load_csv(dev_data_path)
print(bm25_data.shape)

# Import corpus (rag_list.txt)
corpus_path = "./rag_list.txt"
with open(corpus_path, 'r') as f:
    corpus = f.readlines()

## Get corpus, question and answer
#corpus = [text for text in corpus]
#questions = [qst for qst in bm25_data['question, str']]
#answers = [answ for answ in bm25_data['answer, str']]
#
## Preprocess the data
#tokenized_corpus = [bm25.preprocess(text) for text in corpus]
#tokenized_questions = [bm25.preprocess(qst) for qst in questions]
#tokenized_answers = [bm25.preprocess(answ) for answ in answers]

(104, 5)


In [29]:
corpus

["'Ejeren af en ejerlejlighed har sammen med andre ejere af lejligheder ejendomsret til grunden, fælles bestanddele og tilbehør m.v. efter et fordelingstal, der fastsættes som en brøkdel. Til lejligheden hører i samme forhold rettigheder og forpligtelser for ejeren som medlem af ejerforeningen. De i stk. 1 og 2 omhandlede rettigheder og forpligtelser kan ikke adskilles fra ejendomsretten til lejligheden.'",
 "'Højskolen skal følge de af finansministeren fastsatte eller aftalte bestemmelser om løn- og ansættelsesvilkår, herunder om pensionsforhold for højskolens ansatte. Tjenestemandslovens § 21, stk. 4, gælder for medlemmer af højskolens bestyrelse.'",
 "'Uden ugrundet ophold efter repræsentantskabets godkendelse af årsrapporten i henhold til § 22, stk. 1, dog senest 4 måneder efter regnskabsårets udløb, indsendes den reviderede og godkendte årsrapport samt udskrift af revisionsprotokollen vedrørende årsrapportens revision til Beskæftigelsesministeriet og Finanstilsynet. Den indsendte 

**Run the model**

In [28]:
# Create the BM25 model
bm25_model = bm25.init_bm25_corpus(tokenized_corpus)

# Define a list that will store the top 3 paragraphs for each question
bm25_top3 = []

# Get the scores for each question and store them in the list
for i, (question, q) in enumerate(zip(tokenized_questions, questions)):
    _top_3 = bm25_model.get_top_n(question, corpus, n=5) #n is the number of paragraphs to return
    bm25_top3.append([q,_top_3])

bm25_top3[22]

['Hvornår kan Nævnet for Videnskabelig Uredelighed kan behandle sager af egen drift?',
 ['Finanstilsynet kan påbyde et fondsmæglerselskab eller en fondsmæglerholdingvirksomhed at afsætte et medlem af direktionen inden for en frist, der er fastsat af Finanstilsynet, hvis medlemmet af direktionen efter § 75, stk. 1, eller § 76 ikke kan bestride stillingen. Finanstilsynet kan påbyde et medlem af bestyrelsen i et fondsmæglerselskab eller en fondsmæglerholdingvirksomhed at nedlægge sit hverv inden for en frist, der er fastsat af Finanstilsynet, hvis bestyrelsesmedlemmet efter § 75, stk. 1, eller § 76 ikke kan bestride hvervet. Finanstilsynet kan påbyde et fondsmæglerselskab at afsætte en direktør, når der er rejst tiltale mod den pågældende i en straffesag om overtrædelse af straffeloven, den finansielle lovgivning eller anden relevant lovgivning, indtil straffesagen er afgjort, hvis domfældelse vil indebære, at direktøren ikke opfylder kravene i § 75, stk. 1, nr. 3. Varigheden af påbud med

### Create embedding corpus

In [ ]:

def create_embedding_matrix(pooling, save=False, save_folder=None):
    # initialise model
    bert_tokenizer = AutoTokenizer.from_pretrained("KennethTM/bert-base-uncased-danish")
    bert_model = AutoModel.from_pretrained("KennethTM/bert-base-uncased-danish")

    # define device
    device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

    # move model to device
    bert_model.to(device)

    # create list of embedding vectors to concatenate into a torch tensor
    embeddings = []

    # index to track number of mistakes
    idx = 0

    for item in tqdm(rag_list):
        # doing a try and except as some paragraphs may exceed the context window of the BERT (I believe)
        try:
            # tokenize texts
            input_ids = bert_tokenizer.encode(item['text'], return_tensors='pt').to(device)
            # run through BERT
            with torch.no_grad():  # disable gradient computation for inference
                outputs = bert_model(input_ids)
            
            # different kinds of pooling
            if pooling == 'cls':
                embedding_vector = outputs.last_hidden_state[:, 0, :]
            elif pooling == 'max':
                embedding_vector = torch.max(outputs, dim=1)[0]
            elif pooling == 'mean':
                embedding_vector = torch.mean(outputs, dim=1)
            else:
                raise ValueError(f"Unknown pooling method: {pooling}")
            
            # add cls-vector to list of embeddings
            embeddings.append(embedding_vector)
        except:
            # if error then count errors with this
            embeddings.append(torch.zeros(1, 768))
            idx += 1

    print(f'{idx} no. of errors')

    # concatenate list into torch tensor
    embeddings_tensor = torch.cat(embeddings, dim=0)

    if save == True:
        # make sure that folder exists
        os.makedirs(save_folder, exist_ok=True)

        # save tensor 
        torch.save(embeddings_tensor, f'{save_folder}/{pooling}_embeddings_tensor.pt')

    return embeddings_tensor



In [9]:
#create_embedding_matrix(pooling='cls')

### Dense retrieval pipeline

In [10]:
bert_tokenizer = AutoTokenizer.from_pretrained("vesteinn/DanskBERT")
bert_model = AutoModel.from_pretrained("vesteinn/DanskBERT")

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaModel were not initialized from the model checkpoint at vesteinn/DanskBERT and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# you can load cls or max respectively, mean still needs to be created
embeddings_matrix = torch.load('/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/mean_embeddings_DanskBERT.pt')

In [87]:
def dense_retrieval(question, pooling='cls', k=3):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    pooling = 'cls', 'max' or 'mean'
    """
    
    # Encode the input sentence
    input_ids = bert_tokenizer.encode(question, return_tensors="pt")  # Encode and add batch dimension
    # Pass the input through the model
    
    with torch.no_grad():  # disable gradient computation for inference
        outputs = bert_model(input_ids)
    
    if pooling == 'cls':
        # Extract the CLS token representation
        embedding_vector = outputs.last_hidden_state[:, 0, :]
    
    elif pooling == 'max':
        embedding_vector = torch.max(outputs.last_hidden_state, dim=1)[0]

    elif pooling == 'mean':
        embedding_vector = torch.mean(outputs.last_hidden_state, dim=1)
    
    # normalise the cls-embedding and the embedding matrix so that the dot product
    # below is now cosine similarity
    embedding_vector_normalised = embedding_vector / torch.norm(embedding_vector, dim=-1, keepdim=True)
    embeddings_matrix_normalised = embeddings_matrix / torch.norm(embeddings_matrix, dim=-1, keepdim=True)

    # finding most similar vectors with dot product
    dense_retrieval = embeddings_matrix_normalised @ torch.transpose(embedding_vector_normalised, 0, 1)
    
    # sorting retrieved embeddings. this leaves a bunch of nan-values at the top
    sorted_retrieval = torch.sort(dense_retrieval, descending=True, stable=True, dim=0)

    # recreating list, sorting out nan-values
    fixed_retrieval_list = [(item, idx) for (item, idx) in zip(sorted_retrieval[0], sorted_retrieval[1]) if torch.isnan(item) == False]

    # get top k paragraphs
    top_k_indices = [item[1] for item in fixed_retrieval_list[:k]]

    # assemble document 
    document = '\n'.join([rag_list[i]['text'] for i in top_k_indices])

    return document

# check if it works using a random question from the dev set
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(random_question, '\n') 
print(dense_retrieval(random_question, pooling='cls', k=3))


Hvor mange procent må kapitalandele i og lån ydet til en virksomhed eller gruppe af indbyrdes forbundne virksomheder højst udgøre? 

Et pengeinstituts kapitalandele erhvervet for puljemidler, hvor kunderne bærer risikoen, indgår ikke i opgørelsen af fradrag i kapitalgrundlaget.
Priser for ydelser fra en netvirksomhed må ikke stige som følge af det vederlag, der erlægges ved overdragelse af virksomheder.
Værdien af aktier, der i henhold til § 215 i lov om finansiel virksomhed overdrages til en forening, medregnes ikke ved opgørelse af foreningens skattepligtige indkomst.


### Evaluation

### Create columns for retrieved documents

In [40]:
# looks like the dense retrievers are generally retrieving shorter paragraphs

tf_idf_k1 = []
for question in tqdm(dev_set['question, str'], desc='TF-IDF, k=1'):
    tf_idf_k1.append(sparse_retrieval(question, X, k=1))

tf_idf_k2 = []
for question in tqdm(dev_set['question, str'], desc='TF-IDF, k=2'):
    tf_idf_k2.append(sparse_retrieval(question, X, k=2))

tf_idf_k3 = []
for question in tqdm(dev_set['question, str'], desc='TF-IDF, k=3'):
    tf_idf_k3.append(sparse_retrieval(question, X, k=3))

# insert BM25

bm25_k1 = [top_1 for parapgraph in bm25_top5 if (top_1:=parapgraph[0])]
bm25_k2 = [top_2 for parapgraph in bm25_top5 if (top_2:=parapgraph[:2])]
bm25_k3 = [top_3 for parapgraph in bm25_top5 if (top_3:=parapgraph[:-1])]


TF-IDF, k=3: 100%|██████████| 104/104 [00:00<00:00, 147.91it/s]


In [42]:
bm25_k3[2]

["'Uden ugrundet ophold efter repræsentantskabets godkendelse af årsrapporten i henhold til § 22, stk. 1, dog senest 4 måneder efter regnskabsårets udløb, indsendes den reviderede og godkendte årsrapport samt udskrift af revisionsprotokollen vedrørende årsrapportens revision til Beskæftigelsesministeriet og Finanstilsynet. Den indsendte årsrapport skal i det mindste indeholde de obligatoriske bestanddele samt den fulde revisionspåtegning. Sammen med indsendelse af årsrapporten efter stk. 1 indsendes et eksemplar af årsrapporten fra samtlige dattervirksomheder af Arbejdsmarkedets Tillægspension.'",
 'Vedtægterne for et kapitalselskab skal indeholde oplysning om: Kapitalselskabets navn og eventuelle binavne, Kapitalselskabets navn og eventuelle binavne, kapitalselskabets formål, selskabskapitalens størrelse og antallet af kapitalandele eller kapitalandelenes pålydende værdi, kapitalandelenes rettigheder, kapitalselskabets ledelsesorganer, herunder oplysning om den valgte ledelsesstruktur

In [88]:
# loading the embeddings matrix in different cells to save ram
embeddings_matrix = torch.load('/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/cls_embeddings_DanskBERT.pt')

dense_cls_k1 = []
for question in tqdm(dev_set['question, str'], desc='Dense CLS, k=1'):
    dense_cls_k1.append(dense_retrieval(question, pooling='cls', k=1))

dense_cls_k2 = []
for question in tqdm(dev_set['question, str'], desc='Dense CLS, k=2'):
    dense_cls_k2.append(dense_retrieval(question, pooling='cls', k=2))

dense_cls_k3 = []
for question in tqdm(dev_set['question, str'], desc='Dense CLS, k=3'):
    dense_cls_k3.append(dense_retrieval(question, pooling='cls', k=3))

Dense CLS, k=3: 100%|██████████| 106/106 [00:28<00:00,  3.72it/s]


In [92]:
embeddings_matrix = torch.load('/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/max_embeddings_DanskBERT.pt')

dense_max_k1 = []
for question in tqdm(dev_set['question, str'], desc='Dense max, k=1'):
    dense_max_k1.append(dense_retrieval(question, pooling='max', k=1))

dense_max_k2 = []
for question in tqdm(dev_set['question, str'], desc='Dense max, k=2'):
    dense_max_k2.append(dense_retrieval(question, pooling='max', k=2))

dense_max_k3 = []
for question in tqdm(dev_set['question, str'], desc='Dense max, k=3'):
    dense_max_k3.append(dense_retrieval(question, pooling='max', k=3))


Dense max, k=3: 100%|██████████| 106/106 [00:29<00:00,  3.64it/s]


In [93]:
embeddings_matrix = torch.load('/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/mean_embeddings_DanskBERT.pt')

dense_mean_k1 = []
for question in tqdm(dev_set['question, str'], desc='Dense mean, k=1'):
    dense_mean_k1.append(dense_retrieval(question, pooling='mean', k=1))

dense_mean_k2 = []
for question in tqdm(dev_set['question, str'], desc='Dense mean, k=2'):
    dense_mean_k2.append(dense_retrieval(question, pooling='mean', k=2))

dense_mean_k3 = []
for question in tqdm(dev_set['question, str'], desc='Dense mean, k=3'):
    dense_mean_k3.append(dense_retrieval(question, pooling='mean', k=3))

Dense mean, k=3: 100%|██████████| 106/106 [00:28<00:00,  3.73it/s]


In [96]:
# adding to dev set and saving locally
dev_set['tf_idf_k1'] = tf_idf_k1
dev_set['tf_idf_k2'] = tf_idf_k2
dev_set['tf_idf_k3'] = tf_idf_k3
dev_set['dense_cls_k1'] = dense_cls_k1
dev_set['dense_cls_k2'] = dense_cls_k2
dev_set['dense_cls_k3'] = dense_cls_k3
dev_set['dense_max_k1'] = dense_max_k1
dev_set['dense_max_k2'] = dense_max_k2
dev_set['dense_max_k3'] = dense_max_k3
dev_set['dense_mean_k1'] = dense_mean_k1
dev_set['dense_mean_k2'] = dense_mean_k2
dev_set['dense_mean_k3'] = dense_mean_k3

dev_set.to_csv("devset/dev_set.csv", index=False)

### Evaluating neo

In [60]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
MODEL_NAME = "KennethTM/gpt-neo-1.3B-danish"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Set the device
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"
model.to(DEVICE)


GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTNeoBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
          (c_proj):

In [ ]:
neo_answers_tf_idf_k1 = []
# evaluating tf-idf
for question, documents in tqdm(zip(dev_set['question, str'], dev_set['tf_idf_k1']), desc='Answering questions'):

    # assemble a prompt from the documents, question and prompting an answer
    prompt = f"Relevante paragraffer: {documents} Spørgsmål: {question} \nIndsæt svar her baseret på de relevante paragraffer:"

    # tokenize
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)

    max_length = len(input_ids[0]) + 100

    # generate an answer within torch.no_grad() to save compute
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            # generation set to stop at '.' as it otherwise just repeats itself (think it's because we don't sample)
            eos_token_id=tokenizer.encode(' Spørgsmål')[0]
        )

    # decode generated answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip(' Spørgsmål')

    # append answer to list
    neo_answers_tf_idf_k1.append(answer[len(prompt):].strip())  # strip the prompt to leave just the answer

Answering questions: 1it [00:24, 24.59s/it]


KeyboardInterrupt: 

In [117]:
print(dev_set['question, str'].iloc[i])
print(i)
i += 1

Hvad har ejeren af en ejerlejlighed, sammen med andre ejere af lejligheder, ejendoms ret til?
0


In [116]:
i = 0

In [114]:
dev_set['text, str'].iloc[0]

"'Ejeren af en ejerlejlighed har sammen med andre ejere af lejligheder ejendomsret til grunden, fælles bestanddele og tilbehør m.v. efter et fordelingstal, der fastsættes som en brøkdel. Til lejligheden hører i samme forhold rettigheder og forpligtelser for ejeren som medlem af ejerforeningen. De i stk. 1 og 2 omhandlede rettigheder og forpligtelser kan ikke adskilles fra ejendomsretten til lejligheden.'"

In [ ]:
# All the changes to be made

dev_set['question, str'].iloc[0] == 'Hvad har ejeren af en ejerlejlighed, sammen med andre ejere af lejligheder, ejendomsret til?'

